In [4]:
import subprocess
import json

from utils.iter import generate_combinations

In [7]:
def gen_sbatch(config_string):
    sbatch_string = f"""#!/bin/bash

    #SBATCH --nodes=1
    #SBATCH --ntasks-per-node=1
    #SBATCH --cpus-per-task=8
    #SBATCH --time=0:40:00
    #SBATCH --mem=32GB
    #SBATCH --gres=gpu:1
    #SBATCH --job-name=Diffeo
    #SBATCH --mail-type=END
    #SBATCH --mail-user=cm6627@nyu.edu
    #SBATCH --output=slurm/slurm_%j.out
    #SBATCH --error=slurm/slurm_%j.err
    
    module purge
    
    singularity exec --nv \
      --overlay /scratch/cm6627/diffeo_cnn/my_env/overlay-15GB-500K.ext3:ro \
      --overlay /vast/work/public/ml-datasets/imagenet/imagenet-val.sqf:ro \
      --overlay /vast/work/public/ml-datasets/imagenet/imagenet-test.sqf:ro \
      --overlay /vast/work/public/ml-datasets/imagenet/imagenet-train.sqf:ro \
      /scratch/work/public/singularity/cuda12.3.2-cudnn9.0.0-ubuntu-22.04.4.sif \
      /bin/bash -c "source /ext3/env.sh; python fit.py '{config_string}'" """
    return sbatch_string

def submit(config):
    config_string = str(json.dumps(config)).replace('"', '\\"')
    sbatch_string = gen_sbatch(config_string)

    with open("submit.sbatch", "w") as f:
        f.write(sbatch_string)
    
    subprocess.run(["sbatch ./submit.sbatch"], shell=True)

In [8]:
config = {
    "dropout_rate": [0.2, 0.4, 0.6],
    "batch_size": [1, 16, 32, 64],
    "epochs": 1500,
    "optimizer": {
        "name": "AdamW",
        "lr": [1e-3, 1e-4, 1e-5],
        "weight_decay": [0.02, 0.2, 2.0]
    },
    "scheduler": {
        "name": 
    },
    "dataset": "resnet18_imagenet1k_train"
}

for config in generate_combinations(config):
    submit(config)

Submitted batch job 57047666
Submitted batch job 57047667
Submitted batch job 57047668
Submitted batch job 57047669
Submitted batch job 57047670
Submitted batch job 57047671
Submitted batch job 57047672
Submitted batch job 57047673
Submitted batch job 57047674
Submitted batch job 57047675
Submitted batch job 57047676
Submitted batch job 57047677
Submitted batch job 57047678
Submitted batch job 57047679
Submitted batch job 57047680
Submitted batch job 57047681
Submitted batch job 57047682
Submitted batch job 57047683
Submitted batch job 57047684
Submitted batch job 57047685
Submitted batch job 57047686
Submitted batch job 57047687
Submitted batch job 57047688
Submitted batch job 57047689
Submitted batch job 57047690
Submitted batch job 57047691
Submitted batch job 57047692
Submitted batch job 57047693
Submitted batch job 57047694
Submitted batch job 57047695
Submitted batch job 57047696
Submitted batch job 57047697
Submitted batch job 57047698
Submitted batch job 57047699
Submitted batc

In [ ]:
! git add .; git commit -m "sc